In [4]:
from PIL import Image
import numpy as np
import os
import glob
import rawpy
import imageio.v3 as iio
import random

In [ ]:
def load_images_and_exposures(image_paths):
    image_paths = [os.path.join(image_paths, f) for f in os.listdir(image_paths) if f.endswith('.tiff')]
    images = []
    exposures = []
    for p in image_paths:
        img = Image.open(p).convert('L')  # grayscale
        images.append(np.array(img).astype(np.float32))
        
        exposure = float(p.split('/')[-1].split('.')[0])
        exposures.append(1/exposure)
    return np.stack(images), np.array(exposures)

def load_rgb_images(path):
    image_paths = [
        os.path.join(path, f)
        for f in os.listdir(path) 
        if f.endswith('.tiff')
    ]
    exposures = []
    images = []
    for p in sorted(image_paths):
        img = Image.open(p).convert('RGB')
        img_np = np.array(img).astype(np.float32)
        images.append(img_np)
        # If the filename is "0.1.tiff", then exposure_time = 0.1
        filename = os.path.splitext(os.path.basename(p))[0]
        exposure_time = 1/ float(filename)
        exposures.append(exposure_time)
    images = np.stack(images, axis=0)  # shape (P, H, W, 3)
    exposures = np.array(exposures)    # shape (P,)
    return images, exposures


def sample_pixels(images, num_samples=100):
    """Randomly sample pixels from image stack (P, H, W) → (N, P)"""
    P, H, W = images.shape
    samples = []
    for _ in range(num_samples):
        x = random.randint(0, W - 1)
        y = random.randint(0, H - 1)
        samples.append([images[j, y, x] for j in range(P)])
    return np.array(samples, dtype=np.float32)

def sample_rgb_pixels(images, num_samples=100, min_val=12, max_val=243):
    """
    images:    shape (P, H, W, 3) in range [0..255].
    num_samples: how many valid random samples to pick.
    min_val:   ignore any pixel < min_val (approx. 5% brightness).
    max_val:   ignore any pixel > max_val (approx. 95% brightness).
    """
    P, H, W, C = images.shape
    samples_r, samples_g, samples_b = [], [], []
    
    attempts = 0
    max_attempts = 50_000  # limit to avoid infinite loops

    while len(samples_r) < num_samples and attempts < max_attempts:
        x = random.randint(0, W - 1)
        y = random.randint(0, H - 1)
        pixel_series = images[:, y, x, :]  # shape (P,3) across P exposures
        # Check if ALL exposures for R/G/B are within [min_val, max_val]
        if (pixel_series >= min_val).all() and (pixel_series <= max_val).all():
            # separate channels
            r = pixel_series[:, 0]
            g = pixel_series[:, 1]
            b = pixel_series[:, 2]
            samples_r.append(r)
            samples_g.append(g)
            samples_b.append(b)
        attempts += 1

    samples_r = np.array(samples_r, dtype=np.float32)  # (N, P)
    samples_g = np.array(samples_g, dtype=np.float32)
    samples_b = np.array(samples_b, dtype=np.float32)
    return samples_r, samples_g, samples_b


def fit_mitsunaga_nayar(samples, times, degree):
    """
    samples: shape (N, P), pixel values in [0,1].
    times:   shape (P,), actual exposure times (e.g. 0.1, 0.2, 0.4,...).
    degree:  polynomial degree (use smaller to reduce risk of overflow).
    Returns: coefficients c (length degree+1).
    """
    N, P = samples.shape
    M = degree + 1

    A = []
    b = []
    for j in range(P - 1):
        rhs = np.log(times[j]) - np.log(times[j + 1])  # ln(t_j) - ln(t_{j+1})
        for i in range(N):
            z1 = samples[i, j]
            z2 = samples[i, j + 1]
            row = [(z1**m - z2**m) for m in range(M)]
            A.append(row)
            b.append(rhs)

    A = np.array(A, dtype=np.float32)
    b = np.array(b, dtype=np.float32)

    # Solve least squares
    c, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
    return c

def evaluate_response(z, coeffs):
    result = np.zeros_like(z, dtype=np.float32)
    for m, c in enumerate(coeffs):
        result += c * (z ** m)
    return result

def update_exposure_ratios(samples, coeffs, degree):
    N, P = samples.shape
    R_new = []
    for j in range(P - 1):
        total = 0
        for i in range(N):
            z1, z2 = samples[i, j], samples[i, j + 1]
            fz1 = sum(coeffs[m] * (z1 ** m) for m in range(degree + 1))
            fz2 = sum(coeffs[m] * (z2 ** m) for m in range(degree + 1))
            if fz2 > 1e-6:
                total += fz1 / fz2
        R_new.append(total / N)
    return R_new


def compute_fit_error(samples, coeffs, R, degree):
    N, P = samples.shape
    error = 0
    for i in range(N):
        for j in range(P - 1):
            z1, z2 = samples[i, j], samples[i, j + 1]
            fz1 = sum(coeffs[m] * (z1 ** m) for m in range(degree + 1))
            fz2 = sum(coeffs[m] * (z2 ** m) for m in range(degree + 1))
            error += (fz1 - R[j] * fz2) ** 2
    return error

def reconstruct_radiance_rgb(images, times, coeffs_r, coeffs_g, coeffs_b):
    """
    images:  shape (P, H, W, 3), raw images in [0..255]
    times:   shape (P,), exposure times
    coeffs_: polynomial coefficients for R, G, B
    Returns: combined radiance map, shape (H, W, 3).
    """
    P, H, W, _ = images.shape
    radiance = np.zeros((H, W, 3), dtype=np.float32)
    weight_sum = np.zeros((H, W, 3), dtype=np.float32)

    # Convert each image to normalized [0,1]
    images_norm = images / 255.0

    # We'll clamp exponent to avoid overflow
    MAX_EXP = 20.0  # or 25 or 30

    for j in range(P):
        z = images_norm[j]      # shape (H, W, 3)
        ln_t = np.log(times[j]) # log of this exposure time

        # Evaluate polynomial for each channel
        f_r = evaluate_response(z[..., 0], coeffs_r)
        f_g = evaluate_response(z[..., 1], coeffs_g)
        f_b = evaluate_response(z[..., 2], coeffs_b)

        # exp( g(z) - ln(t) ) = exp( g(z) ) / t
        exp_input_r = np.clip(f_r - ln_t, -MAX_EXP, MAX_EXP)
        exp_input_g = np.clip(f_g - ln_t, -MAX_EXP, MAX_EXP)
        exp_input_b = np.clip(f_b - ln_t, -MAX_EXP, MAX_EXP)

        rad_r = np.exp(exp_input_r)
        rad_g = np.exp(exp_input_g)
        rad_b = np.exp(exp_input_b)

        # Weight function (simple "hat")
        w_r = 1.0 - 2.0 * np.abs(z[..., 0] - 0.5)
        w_g = 1.0 - 2.0 * np.abs(z[..., 1] - 0.5)
        w_b = 1.0 - 2.0 * np.abs(z[..., 2] - 0.5)

        w_r = np.clip(w_r, 0.0, 1.0)
        w_g = np.clip(w_g, 0.0, 1.0)
        w_b = np.clip(w_b, 0.0, 1.0)

        radiance[..., 0] += rad_r * w_r
        radiance[..., 1] += rad_g * w_g
        radiance[..., 2] += rad_b * w_b

        weight_sum[..., 0] += w_r
        weight_sum[..., 1] += w_g
        weight_sum[..., 2] += w_b

    # Avoid divide by zero
    mask = (weight_sum < 1e-8)
    weight_sum[mask] = 1.0
    radiance /= weight_sum

    return radiance




def evaluate_response(z, coeffs):
    result = np.zeros_like(z, dtype=np.float32)
    for m, c in enumerate(coeffs):
        result += c * (z ** m)
    return result


def tone_map(img, exposure=1e-3, gamma=2.2):
    mapped = 1.0 - np.exp(-img * exposure)
    mapped = np.clip(mapped, 0, 1)
    mapped = mapped ** (1.0 / gamma)
    return (mapped * 255).astype(np.uint8)



In [105]:
data_path = "../data/raw/"
output_path = "../data/output/"
os.makedirs(output_path, exist_ok=True)

# 1) Load images, exposures
images, times = load_rgb_images(data_path)  

In [106]:
NUM_SAMPLES = 300
samples_r, samples_g, samples_b = sample_rgb_pixels(images, NUM_SAMPLES)
samples_r /= 255.0
samples_g /= 255.0
samples_b /= 255.0

In [107]:
degree = 5  # or whatever you like
coeffs_r = fit_mitsunaga_nayar(samples_r, times, degree)
coeffs_g = fit_mitsunaga_nayar(samples_g, times, degree)
coeffs_b = fit_mitsunaga_nayar(samples_b, times, degree)

In [108]:
radiance_map = reconstruct_radiance_rgb(
    images, 
    times, 
    coeffs_r, 
    coeffs_g, 
    coeffs_b
)

In [110]:
print("HDR min:", radiance_map.min())
print("HDR max:", radiance_map.max())
print("HDR mean:", radiance_map.mean())


HDR min: 0.0
HDR max: 430.68472
HDR mean: 10.129795


In [113]:
tone_mapped = tone_map(radiance_map)
iio.imwrite(os.path.join(output_path, "tone_mapped.jpg"), tone_mapped)

# 7) Optionally store HDR as EXR or HDR
#    imageio can do EXR (if installed with proper plugin).
#    We'll just do 32-bit .hdr for demonstration.
radiance_map_normalized = radiance_map / np.max(radiance_map)
iio.imwrite(os.path.join(output_path, "output.hdr"),
            radiance_map_normalized.astype(np.float32))

In [ ]:
samples_rgb = [samples_r, samples_g, samples_b]
best_coeffs_rgb = []
best_error_rgb = []
best_order_rgb = []



for samples in samples_rgb:
    best_coeffs = None
    best_error = float('inf')
    best_order = None
    for degree in range(1, 8):
        print(f"Trying degree {degree}")
        R = [exposures[j] / exposures[j+1] for j in range(len(exposures) - 1)]
        for _ in range(MAX_ITERS):
            coeffs = fit_polynomial_response(samples, R, degree)
            R = update_exposure_ratios(samples, coeffs, degree)
        error = compute_fit_error(samples, coeffs, R, degree)
        print(f"Degree {degree}, Error = {error:.4f}")
        if error < best_error:
            best_error = error
            best_coeffs = coeffs
            best_order = degree
    best_coeffs_rgb.append(best_coeffs)
    best_error_rgb.append(best_error)
    best_order_rgb.append(best_order)
    
best_coeffs_r, best_coeffs_g, best_coeffs_b = best_coeffs_rgb
best_error_r, best_error_g, best_error_b = best_error_rgb
best_order_r, best_order_g, best_order_b = best_order_rgb


Trying degree 1
Degree 1, Error = 189.4541
Trying degree 2
Degree 2, Error = 413.1399
Trying degree 3
Degree 3, Error = 91.7379
Trying degree 4
Degree 4, Error = 78.6007
Trying degree 5
Degree 5, Error = 80.0019
Trying degree 6
Degree 6, Error = 53.7064
Trying degree 7
Degree 7, Error = 7.7069
Trying degree 8
Degree 8, Error = 101.6240
Trying degree 9
Degree 9, Error = 655.1189
Trying degree 10
Degree 10, Error = 518.7669
Trying degree 1
Degree 1, Error = 334.6911
Trying degree 2
Degree 2, Error = 144.9165
Trying degree 3
Degree 3, Error = 78.9072
Trying degree 4
Degree 4, Error = 100.0462
Trying degree 5
Degree 5, Error = 52.8848
Trying degree 6
Degree 6, Error = 37.2322
Trying degree 7
Degree 7, Error = 32.1803
Trying degree 8
Degree 8, Error = 66.5373
Trying degree 9
Degree 9, Error = 10363.7942
Trying degree 10
Degree 10, Error = 13452.7760
Trying degree 1
Degree 1, Error = 4.2218
Trying degree 2
Degree 2, Error = 154.9901
Trying degree 3
Degree 3, Error = 111.8467
Trying degree 4


In [69]:
radiance_map = reconstruct_radiance_rgb(images, exposures, best_coeffs_r, best_coeffs_g, best_coeffs_b)

In [71]:
tone_mapped = tone_map(radiance_map, exposure=1e-4)
radiance_map_normalized = radiance_map / np.max(radiance_map)
iio.imwrite(os.path.join(output_path, "tone_mapped.jpg"), tone_mapped)
iio.imwrite(os.path.join(output_path, "output.hdr"), radiance_map_normalized.astype(np.float32))